In [ ]:
import sys

sys.path.append("..")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import distributions as dist
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from torchvision.models import vgg11_bn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from pathlib import Path
import pickle
from torch import nn
from inpainting.datasets.mnist import train_val_datasets
from inpainting.visualizations.digits import digit_with_mask as vis_digit_mask
from inpainting.visualizations import visualizations_utils as vis
from inpainting.training import train_inpainter
from inpainting.utils import classifier_experiment, inpainted, predictions_for_entire_loader
from inpainting.datasets import mask_coding as mc
from inpainting.datasets.utils import RandomRectangleMaskConfig
from inpainting.custom_layers import Reshape
from inpainting.losses import nll_masked_batch_loss, nll_masked_batch_loss_components, r2_masked_batch_loss, nll_masked_batch_loss_same_size_masks

# from fancyimpute import NuclearNormMinimization, SoftImpute, BiScaler, MatrixFactorization, IterativeSVD, SoftImpute
from sklearn.impute import KNNImputer#, SimpleImputer, IterativeImputer

In [ ]:
# import cudf, cuml

In [ ]:
ds_train, ds_val = ds_train, ds_val = train_val_datasets(
    Path.home() / "uj/.data/",
    mask_configs_train=[
        RandomRectangleMaskConfig(mc.UNKNOWN_LOSS, 14, 14, 0,0, deterministic=True),
    ],
    mask_configs_val=[
        RandomRectangleMaskConfig(mc.UNKNOWN_LOSS, 14, 14, 0,0),
    ],
    ds_type=MNIST,
    resize_size=(28,28),
)


In [ ]:
for i, ((X, J), y) in enumerate(ds_train):
    X_t = X.numpy()
    J_t = J.numpy()
    
    fig, ax = plt.subplots(1,1)
    
    
#     ax[0].imshow(X_t.reshape(28,28))
    vis.vis_digit_mask(X_t, J_t, ax)
    
    if i > 5:
        break

In [ ]:
mnist_np = np.array([X.numpy() for ((X, J),y) in ds_train]).reshape(-1, 784)

In [ ]:
imputers = [
#     SimpleImputer(),
#     IterativeImputer(),
    KNNImputer(),
#     NuclearNormMinimization(), 
#     SoftImpute(), 
#     BiScaler(), 
#     MatrixFactorization(), 
#     IterativeSVD(), 
#     SoftImpute()
]

In [ ]:
for imp in imputers:
    print(imp)
    imp.fit_transform(mnist_np)

In [ ]:
for i, ((X, J), y) in enumerate(ds_val):
    X_t = X.numpy()
    J_t = J.numpy()
    
    fig, ax = plt.subplots(1, 2+len(imputers))
    
    
    ax[0].imshow(X_t.reshape(28,28))
    vis.vis_digit_mask(X_t, J_t, ax[1])
    X_t[J_t!=1]=np.nan
    
    X_t = X_t.reshape(1, 784)
    
    for j, imp in enumerate(imputers):
        
        X_inp = imp.transform(X_t).reshape(28,28)
        ax[2+j].imshow(X_inp)
        ax[2+j].set_title(str(imp))
    
    plt.show()
    
    if i >= 10:
        break

In [ ]:
df_mean = df.mean()

In [ ]:
df = cudf.DataFrame(data=mnist_np)

In [ ]:
knn = cuml.neighbors.NearestNeighbors(n_neighbors=1)

In [ ]:
knn.fit(df)

In [ ]:
type(knn.X_m)

In [ ]:
np_mean = torch.tensor(np.array(knn.X_m.mean().to_pandas())).reshape(1, 28, 28)

In [ ]:
# plt.imshow(knn.X_m.to_array().reshape(28,28));

In [ ]:
dl = DataLoader(ds_val, batch_size=128, shuffle=False)

In [ ]:

for (X, J), y in dl:
    
    print(X.shape, J.shape)
    
    XJ = X * (J==1) + np_mean
    
    XJ_np = XJ.reshape(XJ.shape[0], -1).cpu().numpy()
    XJ_df = cudf.DataFrame(data=XJ_np)
    
    dist, ind = knn.kneighbors(XJ_df, )
    ind_np = ind.to_array()
    
    
    imputations = torch.tensor(mnist_np[ind_np].reshape(len(ind_np), 1, 28, 28))
    
    X_inp = X * (J==1) + imputations * (J!=1)
    
    

    for i, (x, j, xj, im, xinp) in enumerate(zip(X, J, XJ, imputations, X_inp)):
        fig, ax = plt.subplots(1, 5)
        ax[0].imshow(x.reshape(28,28))
        vis.vis_digit_mask(x, j, ax[1])
        ax[2].imshow(xj.reshape(28,28))
        ax[3].imshow(im.reshape(28,28))
        ax[4].imshow(xinp.reshape(28,28))
        
        if i >= 10:
            break
    
        
    break

In [ ]:
imputations.shape

In [ ]:
ind.to_array()

In [ ]:
knn.X_m.loc[:, ind]

In [ ]:
knn.X_m[ind]

In [ ]:
# plt.imshow(X[0,0])

In [ ]:
# plt.imshow(XJ[0,0])